# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ulsteinvik,62.3432,5.8487,1.84,75,12,2.68,NO,1701973038
1,1,belen,36.4891,36.2233,10.22,100,75,2.57,TR,1701973075
2,2,saipan,15.1355,145.7010,27.39,78,20,6.17,MP,1701972859
3,3,grytviken,-54.2811,-36.5092,0.21,86,70,11.28,GS,1701973076
4,4,iskateley,67.6803,53.1512,-14.43,96,100,3.58,RU,1701973076


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
map_plot1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
new_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
revised_city_df = new_city_df.dropna()

# Display sample data
revised_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,huatabampo,26.8333,-109.6333,26.65,21,0,1.79,MX,1701973080
34,34,lihue,21.9789,-159.3672,21.58,89,0,2.57,US,1701973080
92,92,wailua homesteads,22.0669,-159.3780,21.24,91,0,2.57,US,1701972945
117,117,port elizabeth,-33.9180,25.5701,21.17,88,0,2.06,ZA,1701973095
249,249,sur,22.5667,59.5289,23.39,48,0,1.86,OM,1701973120


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = revised_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
28,huatabampo,MX,26.8333,-109.6333,21,
34,lihue,US,21.9789,-159.3672,89,
92,wailua homesteads,US,22.0669,-159.3780,91,
117,port elizabeth,ZA,-33.9180,25.5701,88,
249,sur,OM,22.5667,59.5289,48,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [18]:
# Set parameters to search for a hotel
radius = 10000
params = {
 "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":100
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    city = row["City"]
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
huatabampo - nearest hotel: No hotel found
lihue - nearest hotel: Kauai Palms
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
port elizabeth - nearest hotel: Waterford Hotel
sur - nearest hotel: Sur Hotel
pisco - nearest hotel: La Portada
abu dhabi - nearest hotel: Marriott Downtown Abu-Dhabi
kapa'a - nearest hotel: Pono Kai Resort
marsa alam - nearest hotel: Blue House Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
28,huatabampo,MX,26.8333,-109.6333,21,No hotel found
34,lihue,US,21.9789,-159.3672,89,Kauai Palms
92,wailua homesteads,US,22.0669,-159.3780,91,Hilton Garden Inn Kauai Wailua Bay
117,port elizabeth,ZA,-33.9180,25.5701,88,Waterford Hotel
249,sur,OM,22.5667,59.5289,48,Sur Hotel
259,pisco,PE,-13.7000,-76.2167,78,La Portada
514,abu dhabi,AE,24.4667,54.3667,53,Marriott Downtown Abu-Dhabi
542,kapa'a,US,22.0752,-159.3190,91,Pono Kai Resort
589,marsa alam,EG,25.0757,34.8918,63,Blue House Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [19]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)